# HTML 파싱(parse)
1. html 페이지를 요청 (`request`)
2. 받아온 html 형식의 문자열을 HTMLElementfh parsing
    - `BeautifulSoup` 라이브러리 활용
3. 선택자(selector, 여러 함수)를 활용해서 text를 추춯

*요청하고 파싱해서 추출하기(특수문자 등이 많아서 복잡하게 보이는 것이지, 과정은 어렵지 않다.)*

## BeautifulSoup import

In [32]:
# 이상한 나라의 앨리스에서 유래
from bs4 import BeautifulSoup

html_txt = """

<div id='container'>
    <div class='wrap-1'>
        <p>Hello HTML</p>
    </div>
    <p class='wrap-2'>
        Bye~
    </p>
</div>

"""

In [9]:
# soup : DOM(Document Object Model)
soup = BeautifulSoup(html_txt, "html.parser") # soup == 위의 html_txt DOM
soup # 더 이상 문자열이 아니다.


<div id="container">
<div class="wrap-1">
<p>Hello HTML</p>
</div>
<p class="wrap-2">
        Bye~
    </p>
</div>

## BeautifulSoup에서 Element를 선택하는 2가지 방법
1. `select`, `select_one` -- 선택자 위주, 간단하고 빠르게 크롤링하고 싶다면
    - `css` 선택자를 활용해서 Element를 선택
    - `select` : 여러 개의 Element를 선택(리스트)
    - `select_one` : 한 개의 Element를 선택(엘리먼트)
2. `find`, `find_all` -- 로직을 껴서 사용하고 싶다면, attribute(선택자)가 복잡하다면
    - 선택자가 아닌, attribute를 이용해 선택할 때 자주 사용한다.
    - `find` : 한 개의 Element가 선택
    - `find_all` : 여러 개의 Element가 선택       

#### `select`, `select_all` 사용하기

In [24]:
p_element = soup.select_one("#container > .wrap-1 > p") # id는 유일한 값이라는 개념부터 시작-> 하위 항목들을 감싸주는 형태 -> 내가 원하는 데이터를 가지고 있는 엘리먼트를 감싸고 있는 엘리먼트를 찾기!!!!!
p_element

<p>Hello HTML</p>

텍스트만 추출하기

In [25]:
p_element.text # .text를 붙이면 문자열 추출

'Hello HTML'

In [14]:
p_elements = soup.select("#container p") # 후손 추출
p_elements

[<p>Hello HTML</p>,
 <p class="wrap-2">
         Bye~
     </p>]

In [22]:
p_elements = soup.select("#container >.wrap-1 > p")
p_elements

[<p>Hello HTML</p>]

In [17]:
p_elements[0].text, p_elements[1].text # 리스트 형식이기 때문에 []를 사용해서 추출

('Hello HTML', '\n        Bye~\n    ')

`find`, `find_all` 사용하기

In [21]:
# div#container > div.wrap-1
soup.find("div", {"id":"container"}).find("div", {"class":"wrap-1"})

<div class="wrap-1">
<p>Hello HTML</p>
</div>

In [26]:
soup.find_all("p")

[<p>Hello HTML</p>,
 <p class="wrap-2">
         Bye~
     </p>]

In [27]:
soup.find("p") # 첫 번째 element만 등장한다

<p>Hello HTML</p>

# Web에서 데이터 불러오기
* naver 환율

In [28]:
NAVER_FINANCE_URL = "https://finance.naver.com/marketindex/"

In [29]:
import requests

# 1. requests 이용해서 HTML 코드 문자열 가져오기
# HTML 페이지는 90% get 방식 통신 (---->-----post방식?---->----셀레니움(최후의 수단. 느림. 브라우저 컨트롤 방식))
response = requests.get(NAVER_FINANCE_URL)
response

<Response [200]>

In [37]:
# 2. BeautifulSoup을 이용하여 css selector를 사용할 수 있는 객체로 파싱
html_text = response.content
soup = BeautifulSoup(html_text, 'html.parser')
soup.select_one("#exchangeList > li.on > a.head.usd > h3")

<h3 class="h_lst"><span class="blind">미국 USD</span></h3>

In [39]:
soup.select_one("#exchangeList > li.on > a.head.usd > div > span.value").text

'1,183.10'

데이터를 묶어서 찾기

In [40]:
li_elem = soup.select_one("#exchangeList > li.on")
li_elem

<li class="on">
<a class="head usd" href="/marketindex/exchangeDetail.naver?marketindexCd=FX_USDKRW" onclick="clickcr(this, 'fr1.usdt', '', '', event);">
<h3 class="h_lst"><span class="blind">미국 USD</span></h3>
<div class="head_info point_up">
<span class="value">1,183.10</span>
<span class="txt_krw"><span class="blind">원</span></span>
<span class="change">0.10</span>
<span class="blind">상승</span>
</div>
</a>
<a class="graph_img" href="/marketindex/exchangeDetail.naver?marketindexCd=FX_USDKRW" onclick="clickcr(this, 'fr1.usdc', '', '', event);">
<img alt="" height="153" src="https://ssl.pstatic.net/imgfinance/chart/marketindex/FX_USDKRW.png" width="295"/>
</a>
<div class="graph_info">
<span class="time">2021.12.06 11:05</span>
<span class="source">하나은행 기준</span>
<span class="count">고시회차<span class="num">102</span>회</span>
</div>
</li>

In [41]:
li_elem.select_one(".h_lst").text

'미국 USD'

In [43]:
li_elem.select_one(".head_info")

<div class="head_info point_up">
<span class="value">1,183.10</span>
<span class="txt_krw"><span class="blind">원</span></span>
<span class="change">0.10</span>
<span class="blind">상승</span>
</div>

In [44]:
li_elem.select_one(".head_info > span.value")

<span class="value">1,183.10</span>

In [45]:
li_elem.select_one(".head_info > .change")

<span class="change">0.10</span>

In [46]:
li_elem.select_one(".head_info > span.blind")
# 자식 선택자 사용

<span class="blind">상승</span>

In [49]:
# 주의!
li_elem.select_one(".head_info span.blind")

# <span class="blind">원</span>
# <span class="blind">상승</span>
# 후손으로 선택한다면, 선택이 두 개가 됨
# 후손 선택자 사용

<span class="blind">원</span>

In [52]:
print(li_elem.select_one(".h_lst").text)
print(li_elem.select_one(".head_info > span.value").text)
print(li_elem.select_one(".head_info > .change").text)
print(li_elem.select_one(".head_info > span.blind").text)

미국 USD
1,183.10
0.10
상승


In [56]:
li_elem_list = soup.select("#exchangeList > li")
len(li_elem_list) # 4종류의 데이터 확인

4

In [58]:
for li_elem in li_elem_list:
    print(li_elem.select_one(".h_lst").text)
    print(li_elem.select_one(".head_info > span.value").text)
    print(li_elem.select_one(".head_info > .change").text)
    print(li_elem.select_one(".head_info > span.blind").text)
    print()
    
#     여러개의 리스트가 있는 경우, 하나만 찾아서 확인하고
#         for문을 써서 붙여넣기

미국 USD
1,183.10
0.10
상승

일본 JPY(100엔)
1,046.99
2.35
상승

유럽연합 EUR
1,336.37
 1.72
하락

중국 CNY
185.72
0.04
상승



데이터 프레임 만들기

In [59]:
naver_finance_dict = {}

naver_finance_dict["통화"] = []
naver_finance_dict["환율"] = []
naver_finance_dict["변동"] = []
naver_finance_dict["등락"] = []

for li_elem in li_elem_list:
    currency = li_elem.select_one(".h_lst").text
    exchange = li_elem.select_one(".head_info > span.value").text
    change = li_elem.select_one(".head_info > .change").text
    updown = li_elem.select_one(".head_info > span.blind").text

    naver_finance_dict["통화"].append(currency)
    naver_finance_dict["환율"].append(exchange)
    naver_finance_dict["변동"].append(change)
    naver_finance_dict["등락"].append(updown)
    
naver_finance_dict    

{'통화': ['미국 USD', '일본 JPY(100엔)', '유럽연합 EUR', '중국 CNY'],
 '환율': ['1,183.10', '1,046.99', '1,336.37', '185.72'],
 '변동': ['0.10', '2.35', ' 1.72', '0.04'],
 '등락': ['상승', '상승', '하락', '상승']}

In [ ]:
1. 메인페이지에서 링크 수집 -> 2. 들어가서 내용 크롤링